# Rf_612 Recover From Invalid Parameters
Likelihood and minimization: Recover from regions where the function is not defined.

We demonstrate improved recovery from disallowed parameters. For this, we use a polynomial PDF of the form
\[
  \mathrm{Pol2} = \mathcal{N} \left( c + a_1 \cdot x + a_2 \cdot x^2 + 0.01 \cdot x^3 \right),
\]

where $ \mathcal{N} $ is a normalisation factor. Unless the parameters are chosen carefully,
this function can be negative, and hence, it cannot be used as a PDF. In this case, RooFit passes
an error to the minimiser, which will try to recover from this situation.

Before ROOT 6.24, RooFit always passed the highest function value that was encountered during the minimisation
to the minimiser. If a parameter is far in a disallowed region, the minimiser has to blindly test various
values of the parameters. It might find the correct values by chance, but often it is simply unable to recover from bad
starting values. Our model here is set up to provoke this behaviour.

Starting with ROOT 6.24, the minimiser receives more information. For example, when a PDF is negative,
the magnitude of the "undershoot" is passed to the minimiser. The minimiser can use this to compute a
gradient, which will eventually lead it out of the disallowed region. The steepness of this gradient
can be chosen using `RooFit::RecoverFromUndefinedRegions(double)`. A value of zero is equivalent to RooFit
before ROOT 6.24. Positive values activate the recovery. Values between 1. and 10. were found to be a
good default. If no argument is passed, RooFit uses 10.


**Author:** Stephan Hageboeck  

## Create a fit model and data
The polynomial is notoriously unstable, because it can quickly go negative.
Since PDFs need to be positive, one often ends up with an unstable fit model.

In [1]:
RooRealVar x("x", "x", -15, 15);
RooRealVar a1("a1", "a1", -0.5, -10., 20.);
RooRealVar a2("a2", "a2", 0.2, -10., 20.);
RooPolynomial pdf("pol3", "c + a1 * x + a2 * x*x + 0.01 * x*x*x", x, RooArgSet(a1, a2));

// Create toy data with an all-positive model:
std::unique_ptr<RooDataSet> dataset(pdf.generate(x, 10000));


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



## Prepare plots
We create a canvas and already plot the data, so we see that we have a PDF model on the verge of going negative.

We also silence RooFit's plotting message streams, because we will later plot an undefined PDF.

In [2]:
%jsroot on  
TCanvas* c = new TCanvas();
RooPlot* frame = x.frame();
dataset->plotOn(frame, RooFit::Name("data"));
frame->Draw();
c->Draw();


RooMsgService::instance().getStream(0).removeTopic(RooFit::Plotting);
RooMsgService::instance().getStream(1).removeTopic(RooFit::Plotting);

# Unstable Fit Models in RooFit before ROOT 6.24
Before 6.24, RooFit wasn't able to recover from invalid parameters.
In this case, the minimiser just errs around the starting values of the fit,
the polynomial is always negative, i.e. the PDF is invalid.

Here, we
- Set up the parameters such that the PDF is negative.
- We run the fit with `RecoverFromUndefinedRegions == 0.`, which is equivalent to what RooFit did in the past.
- When printing the parameters, we see that they are far from `(-0.5, 0.2)`, although the fitter claims that the fit converged.

In [3]:
a1.setVal(10.);
a2.setVal(-1.);

// Perform a fit:
RooFitResult* fitWithoutRecovery = pdf.fitTo(*dataset, RooFit::Save(),
    RooFit::RecoverFromUndefinedRegions(0.), // This is how RooFit behaved prior to ROOT 6.24
    RooFit::PrintEvalErrors(-1), // We are expecting a lot of evaluation errors. -1 switches off printing.
    RooFit::PrintLevel(-1)
);

pdf.plotOn(frame, RooFit::LineColor(kRed), RooFit::Name("noRecovery"));
fitWithoutRecovery->Print();

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization -- RooMinimizer::optimizeConst: deactivating const optimization
[#0] ERROR:Eval -- RooAbsReal::logEvalError(pol3) evaluation error, 
 origin       : RooPolynomial::pol3[ x=x coefList=(a1,a2) ]
 message      : p.d.f normalization integral is zero or negative: -2220.000000
 server values: x=x=0, coefList=(a1 = 2.60781 +/- 11.9002,a2 = -1 +/- 11.5683)

  RooFitResult: minimized FCN value: 0, estimated distance to minimum: 0
                covariance matrix quality: Approximation only, not accurate
                Status : MINIMIZE=0 HESSE=0 

    Floating Parameter    FinalValue +/-  Error   
  --------------------  --------------------------
                    a1    2.6078e+00 +/-  1.19e+01
                    a2   -1.0000e+00 +/-  1.16e+01



# RooFit since ROOT 6.24
In ROOT 6.24, the minimiser gets information about the "badness" of the violation of the function definition range.
In this model, it can see the "undershoot" of the polynomial model.
It uses the magnitude of this undershoot to find its way out of the disallowed parameter regions until the model is positive everywhere.

We first have to reset the parameters, so we start at the same point as above:

In [13]:
a1.setVal(10.);
a2.setVal(-1.);

// Fit again, but pass recovery information to the minimiser:
RooFitResult* fitWithRecovery = pdf.fitTo(*dataset, RooFit::Save(),
    RooFit::RecoverFromUndefinedRegions(1.), // The magnitude of the recovery information can be chosen here.
                                             // Higher values mean more aggressive recovery.
    RooFit::PrintEvalErrors(-1), // We are still expecting a few evaluation errors.
    RooFit::PrintLevel(-1)
 );

pdf.plotOn(frame, RooFit::LineColor(kBlue), RooFit::Name("recovery"));

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization -- RooMinimizer::optimizeConst: deactivating const optimization


## Collect results and plot
We print the two fit results, and plot the fitted curves.
The curve of the fit without recovery cannot be plotted, because the PDF is undefined if a2 < 0.
 
- Without recovery, the fitter encounters ~70 invalid function values, but it claims that the fit converged.
  Note: the minimised function value is 0, and the parameters are wrong. We expect (-0.5, 0.2).
- With recovery (default in ROOT 6.24) it finds more points with invalid parameters, but this is because it's
  computing a way out of this region. The function is minimised correctly.

In [14]:
fitWithoutRecovery->Print();
std::cout << fitWithoutRecovery->numInvalidNLL() << std::endl;

fitWithRecovery->Print();
fitWithRecovery->numInvalidNLL()


  RooFitResult: minimized FCN value: 0, estimated distance to minimum: 0
                covariance matrix quality: Approximation only, not accurate
                Status : MINIMIZE=0 HESSE=0 

    Floating Parameter    FinalValue +/-  Error   
  --------------------  --------------------------
                    a1    2.6078e+00 +/-  1.19e+01
                    a2   -1.0000e+00 +/-  1.16e+01

66

  RooFitResult: minimized FCN value: 30139.3, estimated distance to minimum: 1.01704e-06
                covariance matrix quality: Full, accurate covariance matrix
                Status : MINIMIZE=0 HESSE=0 

    Floating Parameter    FinalValue +/-  Error   
  --------------------  --------------------------
                    a1   -4.4830e-01 +/-  4.05e-02
                    a2    1.8274e-01 +/-  1.71e-02

(int) 169


Finalise the plot. Note that the red curve is constant at zero, since it is always negative, which is impossible for a PDF.

In [15]:
TLegend* legend = new TLegend(0.5, 0.7, 0.9, 0.9);
legend->SetBorderSize(0);
legend->SetFillStyle(0);
legend->AddEntry(frame->findObject("data"), "Data", "P");
legend->AddEntry(frame->findObject("noRecovery"), "Without recovery (cannot be plotted)", "L");
legend->AddEntry(frame->findObject("recovery"), "With recovery", "L");
frame->Draw();
legend->Draw();
c->Draw();